In [6]:
import re
import dateparser
from transformers import pipeline
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# lets load the data
loader = TextLoader(r"C:\Users\User\Desktop\Faiss\Faiss data.txt", encoding="utf-8")
documents = loader.load()

# split the documents into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

#lets transfrom the chunks of text into meaningful embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
#now we save the embeddings into a vector database
vector_db = FAISS.from_documents(docs, embeddings)

#pipeline for summerization and question answering
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_model = pipeline("text2text-generation", model="google/flan-t5-small")

#function to check call request in query
def is_call_request(query):
    return any(kw in query.lower() for kw in ["call me", "contact me", "reach me"])
#function to check appointment request in query
def is_appointment_request(query):
    return any(kw in query.lower() for kw in ["book", "appointment", "schedule", "meeting"])

#mail validation in format something + @ something + . + somthing
def validate_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email)
 #mail validation ^ means start $ means the end \+ means + and ? at last says + can be optional and then \d{7,15} numbers can be between 7 to 15
def validate_phone(phone):
    return re.match(r"^\+?\d{7,15}$", phone)
 
 #funntion to extract date from natural language, remember its a rule based parser, so it might not work for all cases
def extract_date(text):
    date = dateparser.parse(text, settings={"PREFER_DATES_FROM": "future"})
    return date.strftime("%Y-%m-%d") if date else None

# user contact colletion
def collect_user_info():
    print("\n Let's collect your contact info.")
    name = input(" What's your name? ").strip()
    while not name:
        name = input("Please enter a valid name: ").strip()

    phone = input(" Your phone number? ").strip()
    while not validate_phone(phone):
        phone = input("Invalid phone number. Try again: ").strip()

    email = input(" Your email address? ").strip()
    while not validate_email(email):
        email = input("Invalid email. Try again: ").strip()

    print("\n Thank you! We will contact you soon.")
    print(f" Name: {name}\n Phone: {phone}\n Email: {email}")

def collect_appointment_info():
    print("\n Let's schedule your appointment.")
    while True:
        date_input = input("When would you like to book? (e.g., 'next Friday')\n> ")
        parsed = extract_date(date_input)
        if parsed:
            break
        print("Couldn't parse date. Try again.")

    name = input(" Your name? ").strip()
    while not name:
        name = input("Please enter a valid name: ").strip()

    phone = input(" Your phone number? ").strip()
    while not validate_phone(phone):
        phone = input("Invalid phone number. Try again: ").strip()

    email = input(" Your email address? ").strip()
    while not validate_email(email):
        email = input("Invalid email. Try again: ").strip()

    print("\n Appointment booked!")
    print(f" Date: {parsed}\n Name: {name}\n Phone: {phone}\n Email: {email}")

# Function to answer user queries based on the context with is the closest chunk to the query
def answer_query(query):
    relevant_docs = vector_db.similarity_search(query, k=3)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    prompt = f"Answer the question based on the context below.\n\nContext:\n{context}\n\nQuestion: {query}"
    result = qa_model(prompt, max_length=256, do_sample=False)
    return result[0]['generated_text']

# Here comes our chatbot loop
print("\n Welcome to your AI Assistant, you can ask ay question, book an appointment, or request a call.")
while True:
    query = input("\n Your input (or 'exit'): ")
    if query.lower() == "exit":
        break

    if is_call_request(query):
        collect_user_info()
        continue

    if is_appointment_request(query):
        collect_appointment_info()
        continue

    answer = answer_query(query)
    print(" Answer:", answer)


Device set to use cpu
Device set to use cpu



 Welcome to your AI Assistant, you can ask ay question, book an appointment, or request a call.


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: AI is used for fraud detection, algorithmic trading, and risk assessment


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: bad

 Let's schedule your appointment.
Couldn't parse date. Try again.
Couldn't parse date. Try again.
Couldn't parse date. Try again.
Couldn't parse date. Try again.

 Appointment booked!
 Date: 2025-07-25
 Name: ankit sapkota
 Phone: 9823456789
 Email: adsaa@af.com


Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: AI has become a transformative force across various industries, from healthcare and finance to agriculture and education. Its ability to process massive amounts of data, identify patterns, and make intelligent predictions has revolutionized the way we interact with technology. One of the key branches of AI is machine learning, which allows systems to improve their performance over time without being explicitly programmed
